In [10]:
import os
from collections import namedtuple
from bbchallenge import *
from typing import List
import tqdm

In [11]:
DB_PATH = "all_5_states_undecided_machines_with_global_header"

# A simple heuristic to detect bouncers

In [25]:
TapeExtreme = namedtuple('TapeExtreme', ['time', 'side', 'pos'])

def simulate_and_get_extremes(machine, time_limit = 10000):
    
    curr_time = 0
    curr_state = 0
    curr_pos = 0
    tape = {}
    
    min_pos, max_pos = 0,0
    tape_extremes: List[TapeExtreme] = []

    while curr_state != None and curr_time < time_limit:
        
        if curr_time == 0 or curr_pos <= min_pos:
            min_pos = curr_pos
            tape_extremes.append(TapeExtreme(curr_time,"left",curr_pos))
            
        if curr_time == 0 or curr_pos >= max_pos:
            max_pos = curr_pos
            tape_extremes.append(TapeExtreme(curr_time,"right",curr_pos))
        
        curr_state, curr_pos = step(machine, curr_state, curr_pos, tape)
       
        curr_time += 1
        
    return tape_extremes

In [13]:
def get_local_extremes(extremes):
    to_return = []
    for i in range(len(extremes)):
        
        if i < len(extremes) - 1 and extremes[i].side == extremes[i+1].side:
            continue
        
        to_return.append(extremes[i])
    
    return to_return

In [46]:
def get_successive_tape_sizes(locally_extremes):
    if len(locally_extremes) < 3:
        return []
    
    to_return = []
    for i in range(2,len(locally_extremes),2):
        to_return.append(locally_extremes[i-1].pos - locally_extremes[i].pos+1)
    
    return to_return     

In [47]:
def discrete_differences(seq):
    if len(seq) <= 1:
        return []
    to_ret = []
    
    for i in range(1,len(seq)):
        to_ret.append(seq[i]-seq[i-1])
        
    return to_ret

In [48]:
def only_zeros(l: List[int]):
    for e in l:
        if e != 0:
            return False
    return True

In [49]:
def heuristic_bouncers(machine, time_limit = 10000, number_of_points_to_conclude = 25):
    extremes = simulate_and_get_extremes(machine)
    local_extremes = get_local_extremes(extremes)
    successive_tape_sizes = get_successive_tape_sizes(local_extremes)
    second_difference = discrete_differences(discrete_differences(successive_tape_sizes))
    print(successive_tape_sizes)
    if len(second_difference) >= number_of_points_to_conclude and \
       only_zeros(second_difference[-1*number_of_points_to_conclude:]):
        return True
    
    return False

## Test on 1 machine

In [50]:
machine = get_machine_i(DB_PATH, 58329156)
heuristic_bouncers(machine)

[5, 9, 13, 17, 21, 25, 29, 33, 37, 41, 45, 49, 53, 57, 61, 65, 69, 73, 77, 81, 85, 89, 93, 97, 101, 105, 109, 113, 117, 121, 125, 129, 133, 137, 141, 145, 149, 153, 157, 161, 165, 169, 173, 177, 181, 185, 189, 193, 197]


True

## Running on all the index of undecided machines

In [66]:
UNDECIDED_INDEX_PATH= "bb5_undecided_index"#_with_heuristics"
UNDECIDED_FILE_SIZE = os.path.getsize(UNDECIDED_INDEX_PATH)

In [67]:
undecided_index = []
with open(UNDECIDED_INDEX_PATH, "rb") as f:
    for i in range(UNDECIDED_FILE_SIZE//4):
        chunk = f.read(4)
        undecided_index.append(int.from_bytes(chunk, byteorder="big"))

In [69]:
heuristically_decided_machines_id = []
i = 0
for machine_id in tqdm.tqdm_notebook(undecided_index):
    machine = get_machine_i(DB_PATH, machine_id)
    if heuristic_bouncer(machine):
        heuristically_decided_machines_id.append(machine_id)
        
    i += 1
    
    if i % 10000 == 0:
        print(i, len(heuristically_decided_machines_id))

<ipython-input-69-56af4b8cd32b>:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for machine_id in tqdm.tqdm_notebook(undecided_index):


  0%|          | 0/1538612 [00:00<?, ?it/s]

10000 7236
20000 14280
30000 21334
40000 28560
50000 35738
60000 42750
70000 50162
80000 57193
90000 64365
100000 71122
110000 77667
120000 84583
130000 91593
140000 98656
150000 105337
160000 111688
170000 118026
180000 124646
190000 130513
200000 137368


KeyboardInterrupt: 